# Compile and deploy pretrained SSD mobilenet model on Jetson TX2

The purpose for this project is to use Neo to compile a pretrained model and then use Greengrass to deploy on Jetson TX2.

## Setup

Before started, please setup Jetson TX2 following https://alpha-docs-aws.amazon.com/greengrass/latest/developerguide/setup-filter.other.html. 
After setting up environment for TX2, following [AWS IoT Greengrass Software Download Links](https://alpha-docs-aws.amazon.com/greengrass/latest/downloadlinks/gg-software.html) to download [AWS IoT Greengrass Core Software](https://alpha-docs-aws.amazon.com/greengrass/latest/developerguide/lambda-functions.html#lambda-sdks-core) and install properly on device.

To compile the Object Detection model on Amazon SageMaker, we also need to setup and authenticate the use of AWS services. To begin with we need an AWS account role with SageMaker access. This role is used to give SageMaker access to your data in S3 will automatically be obtained from the role used to start the notebook.

In [ ]:
import boto3
import sagemaker
import time
import json
from sagemaker.utils import name_from_base
from sagemaker import get_execution_role 
from sagemaker.model import NEO_IMAGE_ACCOUNT
from sagemaker.fw_utils import create_image_uri

In [ ]:
role = get_execution_role() 
print(role)

Go to AWS service console and then search for `IAM`. Click on `Role` on the left sidebar, find the above role, attach the following policies to that role.

- AWSGreengrassFullAccess
- AWSGreengrassResourceAccessRolePolicy
- AWSIoTFullAccess
- AWSLambdaRole 

**Note: Policy ARN of `AWSLambdaRole` will be used later when create lambda function**

In [ ]:
sess = sagemaker.Session()
region = sess.boto_region_name
account = sess.boto_session.client('sts').get_caller_identity()['Account']
framework = 'mxnet'
framework_version = '1.4.1'
bucket = sess.default_bucket()
prefix = 'your_prefix'

Amazon SageMaker provides prebuilt Docker images that include deep learning framework libraries and other dependencies needed for training and inference. Check here for a list of available [SageMaker container images](https://docs.aws.amazon.com/sagemaker/latest/dg/neo-deployment-hosting-services-console.html).

In [ ]:
# target device is not the device we are using here, it's a aws instance, which the container image was build on
target_device = 'ml_p3'
image_url = create_image_uri(region, 
                             'neo-' + framework.lower(), 
                             target_device.replace('_', '.'),
                             framework_version, 
                             py_version='py3', 
                             account=NEO_IMAGE_ACCOUNT[region]
                            )
print(image_url)

## The Preprocess and Postprocess Script

The `entry_point.py` script provides the preprocess and postprocess methods that we need to correctly process the input image.

In [ ]:
!cat entry_point.py

## Import pretrained mobilenet model

First we need to get the pretrained model from model zoo, then we should compress the `model.json` and `model.params`  and upload the tarball to a S3 bucket for the SageMaker Python SDK to compile.

You can read more about creating an `MXNetModel` object in the [SageMaker Python SDK API docs](https://sagemaker.readthedocs.io/en/stable/sagemaker.mxnet.html#mxnet-model).

### Download pretrained model
Use the `get_model.py` script to download pretrained `ssd_512_mobilenet1.0_voc` model from gluoncv model zoo.

In [ ]:
!python3 get_model.py

Compress the `*.pramas` and `*.json` files downloaded from previous step to a tarball

In [ ]:
!tar -zcvf mobilenet.tar.gz mobilenet-0000.params mobilenet-symbol.json

### Upload model to S3 bucket.

In [ ]:
import logging
import boto3
from botocore.exceptions import ClientError

def upload_file(file_name, bucket, object_name=None):
    """Upload a file to an S3 bucket

    :param file_name: File to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name. If not specified then file_name is used
    :return: True if file was uploaded, else False
    """

    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = file_name

    # Upload the file
    s3_client = boto3.client('s3')
    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [ ]:
upload_file('mobilenet.tar.gz', bucket)

In [ ]:
from sagemaker.mxnet.model import MXNetModel
from sagemaker.predictor import RealTimePredictor

mobilenet_model = MXNetModel(model_data='s3://path/to/your/model,
                             image=image_url,
                             entry_point='entry_point.py',
                             predictor_cls=RealTimePredictor,
                             role=role,
                             sagemaker_session=sess,
                             py_version='py3',
                             framework_version=framework_version
                            )

## Compile pretrained model for Jetson TX2

Deploy using Neo API to optimize the model performance for your target device, here I use Jetson Tx2. 

In [ ]:
output_path = 's3://{}/{}'.format(bucket, prefix)
compiled_mobilenet = mobilenet_model.compile(target_instance_family='jetson_tx2', 
                                             input_shape={'data':[1,3,512,512]},
                                             job_name='your-job-name',
                                             role=role,
                                             framework=framework,
                                             output_path=output_path
                                            )

# Deploy compiled model on Jetson TX2

After we have an endpoint for the model to perform inference, we will deploy the endpoint to target device and then inference with this device.

## Use Greengrass API to deploy a Neo compiled model on edge device

Please refer to [What is AWS IoT Greengrass](https://alpha-docs-aws.amazon.com/greengrass/latest/developerguide/what-is-gg.html) to learn more.

To use Greengrass API, we need to create clients from the default boto3 session.

In [ ]:
GG = boto3.client('greengrass')

In [ ]:
IoT = boto3.client('iot')

#### Create a Greengrass group and configure its core

In [ ]:
group = GG.create_group(Name='final_ssd_mobilenet_tx2_demo')

In [ ]:
group

Create keys and certificate for group. Keys and cerificate should be place in the right place on device to ensure communication between device and Greengrass.

In [ ]:
keys_cert = IoT.create_keys_and_certificate(setAsActive=True)

Create a core for group. There can be only one core for each group.

In [ ]:
core_thing = IoT.create_thing(thingName=group['Name']+"_core")

In [ ]:
core_thing

Attach policy to core

In [ ]:
IoT.attach_thing_principal(thingName=core_thing['thingName'], principal=keys_cert['certificateArn'])

In [ ]:
core_policy_doc = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "iot:Publish",
                "iot:Subscribe",
                "iot:Connect",
                "iot:Receive",
                "iot:GetThingShadow",
                "iot:DeleteThingShadow",
                "iot:UpdateThingShadow"
            ],
            "Resource": ["arn:aws:iot:" + boto3.session.Session().region_name + ":*:*"]
        },
        {
            "Effect": "Allow",
            "Action": [
                "greengrass:AssumeRoleForGroup",
                "greengrass:CreateCertificate",
                "greengrass:GetConnectivityInfo",
                "greengrass:GetDeployment",
                "greengrass:GetDeploymentArtifacts",
                "greengrass:UpdateConnectivityInfo",
                "greengrass:UpdateCoreDeploymentStatus"
            ],
            "Resource": ["*"]
        }
    ]
}

In [ ]:
policy = IoT.create_policy(
    policyName="my_core_policy_clone1",
    policyDocument=json.dumps(core_policy_doc)
)

In [ ]:
IoT.attach_principal_policy(
    policyName=policy['policyName'],
    principal=keys_cert['certificateArn']
)

In [ ]:
initial_version = {'Cores': []}
initial_version['Cores'].append(
    {
        'Id': core_thing['thingName'], # Quite intuitive, eh?
        'CertificateArn': keys_cert['certificateArn'],
        'SyncShadow': False, # Up to you, True|False
        'ThingArn': core_thing['thingArn']
    }
)

In [ ]:
initial_version

`create_core_definition()`Grants permission to create a version of an existing core definition. Greengrass groups must each contain exactly one Greengrass core.

In [ ]:
core_definition = GG.create_core_definition(
    Name="{0}_core_def".format(group['Name']),
    InitialVersion=initial_version
)

In [ ]:
core_definition

#### Create a group version

In [ ]:
group_ver = GG.create_group_version(
    GroupId=group['Id'],
    CoreDefinitionVersionArn=core_definition['LatestVersionArn']
)

In [ ]:
group_ver

In [ ]:
print ("Group: https://{}.console.aws.amazon.com/iot/home?region={}#/greengrass/groups/{}".format(region, region, group['Id']))
print ("Core thing: https://{0}.console.aws.amazon.com/iot/home?{0}#/thing/my_group_core_1".format(region))
print ("Certificate and policy: https://{0}.console.aws.amazon.com/iot/home?region={0}#/certificate/{1}".format(region, keys_cert['certificateArn']))

In [ ]:
groupParam = {
    'group': group,
    'core_thing': core_thing,
    'keys_cert': keys_cert,
    'group_ver': group_ver,
    'core_definition': core_definition,
    'policy': policy
}
with open('./final_demo.json', 'w') as f:
    json.dump(groupParam, f, indent=4)

In [ ]:
groupParam

From the groupParam.json, export the certificate Pem and key pairs, we will copy these files to Jetson TX2 later.

In [ ]:
certPem = keys_cert['certificatePem']
with open('./myCore.cert.pem', 'w') as f:
    for line in certPem.split('\n'):
        f.write(line)
        f.write('\n')

In [ ]:
public = keys_cert['keyPair']['PublicKey']
with open('./myCore.public.key', 'w') as f:
    for line in public.split('\n'):
        f.write(line)
        f.write('\n')

In [ ]:
private = keys_cert['keyPair']['PrivateKey']
with open('./myCore.private.key', 'w') as f:
    for line in private.split('\n'):
        f.write(line)
        f.write('\n')

In [ ]:
config = {
  "coreThing" : {
    "caPath" : "root.ca.pem",
    "certPath" : "myCore.cert.pem",
    "keyPath" : "myCore.private.key",
    "thingArn" : core_thing["thingArn"],
    "iotHost" : "a3lhyy9ngfgeht-ats.iot.us-west-2.amazonaws.com",
    "ggHost" : "greengrass-ats.iot.us-west-2.amazonaws.com",
    "keepAlive" : 600
  },
  "runtime" : {
    "cgroup" : {
      "useSystemd" : "yes"
    }
  },
  "managedRespawn" : False,
  "crypto" : {
    "principals" : {
      "SecretsManager" : {
        "privateKeyPath" : "file:///greengrass/certs/myCore.private.key"
      },
      "IoTCertificate" : {
        "privateKeyPath" : "file:///greengrass/certs/myCore.private.key",
        "certificatePath" : "file:///greengrass/certs/myCore.cert.pem"
      }
    },
    "caPath" : "file:///greengrass/certs/root.ca.pem"
  }
}

with open('./config.json', 'w') as f:
    json.dump(config, f, indent=4)

On device, under the greengrass folder, create a folder called certs, then, download the appropriate ATS root CA certificate. The following example downloads `AmazonRootCA1.pem`. 

`
cd /greengrass
mkdir certs
mkdir config
cd certs
sudo wget -O root.ca.pem https://www.amazontrust.com/repository/AmazonRootCA1.pem`

Copy the certificate and keys to certs folder. Copy `config.json` to config folder. 

Put certificates in place, adjust `config.json` according to your keys' name, and then launch the Greengrass daemon.

#### Create a Resource Definition and Version

In [ ]:
resource = GG.create_resource_definition(
    InitialVersion={
        'Resources': [
            {
                'Id': 'my-ml-resource',
                'Name': 'my-ml-resource',
                'ResourceDataContainer': {
                    'S3MachineLearningModelResourceData': {
                        'DestinationPath': '/ml_model',
                        'S3Uri': 'https://{}.s3-{}.amazonaws.com/{}'.format(bucket, region, output_path.split('//')[1])
                    }
                }
            }
        ]
    }
)

In [ ]:
resource

#### Create a Lambda Function Deployment Package
Before creating a Lambda function, following [AWS IoT Greengrass Software Download Links](https://alpha-docs-aws.amazon.com/greengrass/latest/downloadlinks/gg-software.html) to download AWS IoT Greengrass Core SDK Software and AWS IoT Greengrass ML SDK Software to your local machine. Unzip the downloaded package to get the SDK. The SDKs are the `greengrasssdk` and `greengrass_machine_learning_sdk` folder.

We also need to create a inferece file. The inference file is very similiar to the usage example in https://docs.aws.amazon.com/greengrass/latest/developerguide/obj-detection-connector.html

Zip the following items into a file named *ssd_mobilenet_mxnet_python_lambda.zip*. When creating the ZIP file, include only the code and dependencies, not the containing folder.

- **inference.py**

- **greengrasssdk**

- **greengrass_machine_learning_sdk**

- **your_test_img.jpg**

This is your Lambda function deployment package.

You can directly use S3 console to upload the Lambda function deployment package to the same S3 bucket which used to store the pretrained mobilenet model.

In [ ]:
Lambda = boto3.client('lambda')

#### Create a Lambda Function

In [ ]:
lambda_fn = Lambda.create_function(
    FunctionName='object_detection_tx2_lambda_final_demo_clone2',
    Runtime='python2.7',
    Role="arn:aws:iam::886656810413:role/Lambda-Role",#the AWSLambdaRole create at the begining of this tutorial
    Handler='inference.function_handler',
    Code={
        'S3Bucket': bucket,
        'S3Key': 'ssd_mobilenet_mxnet_python_lambda.zip'
    },
    Timeout=300,
    MemorySize=2048,
    Publish=True
)

In [ ]:
lambda_fn

#### Create an alias for the lambda function

In [ ]:
alias = Lambda.create_alias(
    FunctionName=lambda_fn['FunctionName'],
    Name='tx2_demo',
    FunctionVersion='1'
)

In [ ]:
alias

#### Create a Function Definition and Version

In [ ]:
lambda_fn = GG.create_function_definition(
    InitialVersion={
        'DefaultConfig': {
            'Execution': {
                'IsolationMode': 'GreengrassContainer'
            }
        },
        'Functions': [
            {
                'FunctionArn': alias['AliasArn'],
                'FunctionConfiguration': {
                    'EncodingType': 'binary',
                    'Environment': {
                        'AccessSysfs': True,
                        'Execution': {
                            'IsolationMode': 'GreengrassContainer',
                        },
                        'ResourceAccessPolicies': [
                            {
                                'Permission': 'rw',
                                'ResourceId': 'my-ml-resource'
                            },
                        ]
                    },
                    'Executable': 'inference.py',
                    'MemorySize': 500000,
                    'Pinned': True,
                    'Timeout': 300
                },
                'Id': 'ObjectDetection'
            },
        ]
    },
    Name='objectDetection' #the name should match the ServiceName inside lambda function
)

In [ ]:
lambda_fn

#### Create a Connector Definition and Version. 

In this project, we use a ML Object Detection connector to perform object detection. The ML Object Detection connectors are bundled with the Amazon SageMaker Neo deep learning runtime (DLR). The connectors use the runtime to serve the ML model. To use these connectors, you must install the dependencies for the DLR on your core device. [Installing Neo Deep Learning Runtime Dependencies on the AWS IoT Greengrass Core](https://alpha-docs-aws.amazon.com/greengrass/latest/developerguide/obj-detection-connector.html#obj-detection-connector-config)

In [ ]:
connector = GG.create_connector_definition(
    InitialVersion={
        'Connectors':[
            {
                "Id": "ObjectDetectionConnectorTX2",
                #object detection connector for tx2
                "ConnectorArn": "arn:aws:greengrass:{}::/connectors/ObjectDetectionAarch64JTX2/versions/1".format(region), 
                "Parameters": {
                    "MLModelDestinationPath": "/ml_model",
                    "MLModelResourceId": "my-ml-resource",
                    "LocalInferenceServiceName": "objectDetection",
                    "LocalInferenceServiceTimeoutSeconds": "300", 
                    "LocalInferenceServiceMemoryLimitKB": "500000",
                    "GPUAcceleration": "GPU"
                }
            }
        ]
    }
)
connector

#### Update the group version

In [ ]:
group_ver = GG.create_group_version(
    ConnectorDefinitionVersionArn=connector['LatestVersionArn'],
    CoreDefinitionVersionArn=core_definition['LatestVersionArn'],
    FunctionDefinitionVersionArn=lambda_fn['LatestVersionArn'],
    GroupId=group['Id'],
    ResourceDefinitionVersionArn=resource['LatestVersionArn']
)

In [ ]:
group_ver

#### Create a deployment
Before deploy, make sure the greengrass deamon is running on device.

`ps aux | grep -E 'greengrass.*daemon'`

In [ ]:
deploy = GG.create_deployment(
    DeploymentType='NewDeployment',
    GroupId=group['Id'],
    GroupVersionId=group_ver['Version']
)

In [ ]:
deploy

#### Get the deployment status.
The deployment may take a few minutes, check the status periodically until success.

In [ ]:
while True:
    status = GG.get_deployment_status(
        DeploymentId=deploy['DeploymentId'],
        GroupId=group['Id']
    )
    if status['DeploymentStatus'] == 'Success':
        break
    elif status['DeploymentStatus'] == 'Failed':
        raise RuntimeError('Deployment Failed')
    print('Deploy ...')
    time.sleep(30)
print('Done!')